# Step-by-step Plan

In [ ]:
!pip install -q \
    "torch==2.1.2" \
    "transformers==4.39.3" \
    "datasets==2.14.6" \
    "torchmetrics==1.3.1" \
    "fsspec==2023.10.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 120.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 121.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Step 2: Load and Tokenize the Dataset

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load the simplified version of the GoEmotions dataset
dataset = load_dataset("go_emotions", "simplified")

# Load DistilBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenization function
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

# Apply tokenizer
tokenized = dataset.map(tokenize, batched=True)

# Format for PyTorch
tokenized.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

# Step 3: Define the Model

In [ ]:
import torch
from torch import nn
from transformers import DistilBertModel

# Number of classes in GoEmotions simplified version
NUM_CLASSES = 28

class DistilBertMultiLabel(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, NUM_CLASSES)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]
        pooled_output = self.dropout(pooled_output)
        return self.classifier(pooled_output)

# Step 4: Create DataLoaders (for training & validation)

In [ ]:
from torch.utils.data import DataLoader

# Split into train and validation
train_data = tokenized["train"]
val_data   = tokenized["validation"]

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=16)

# Step 5: Initialize Model, Loss, Optimizer, and Metric

In [ ]:
import torch.optim as optim
from torch.nn import BCEWithLogitsLoss
from transformers import logging
import torchmetrics

logging.set_verbosity_error()  # silence warning spam

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = DistilBertMultiLabel().to(device)

# For multi-label classification
criterion = BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

# F1 score metric
f1_metric = torchmetrics.classification.MultilabelF1Score(
    num_labels=28, average="weighted").to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

# Create a one‑hot helper and custom collate

In [ ]:
tokenized.reset_format()          # remove Torch formatting completely

In [ ]:
from torch.utils.data import Dataset
import torch

NUM_CLASSES = 28  # simplified GoEmotions

class GoEmotionDataset(Dataset):
    def __init__(self, hf_ds):
        self.ds = hf_ds

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        ex = self.ds[idx]
        # convert input_ids & mask to torch tensors
        item = {
            "input_ids": torch.tensor(ex["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(ex["attention_mask"], dtype=torch.long),
            "labels": ex["labels"]           # keep as python list for now
        }
        return item

# Rebuild DataLoaders with that collate_fn


In [ ]:
def multi_hot(label_list):
    vec = torch.zeros(NUM_CLASSES, dtype=torch.float32)
    vec[label_list] = 1.0
    return vec

def collate_fn(batch):
    ids   = torch.stack([b["input_ids"] for b in batch])
    mask  = torch.stack([b["attention_mask"] for b in batch])
    labels = torch.stack([multi_hot(b["labels"]) for b in batch])
    return {"input_ids": ids, "attention_mask": mask, "labels": labels}

# Re‑run the training loop

In [ ]:
train_loader = DataLoader(
    GoEmotionDataset(tokenized["train"]), batch_size=16, shuffle=True, collate_fn=collate_fn)

val_loader = DataLoader(
    GoEmotionDataset(tokenized["validation"]), batch_size=32, shuffle=False, collate_fn=collate_fn)

# Run the same training loop again

In [ ]:
for epoch in range(1):
    model.train(); total = 0
    for batch in train_loader:
        ids   = batch["input_ids"].to(device)
        mask  = batch["attention_mask"].to(device)
        y     = batch["labels"].to(device)

        optimizer.zero_grad()
        loss = criterion(model(ids, mask), y)
        loss.backward()
        optimizer.step()
        total += loss.item()

    print(f"Train loss: {total/len(train_loader):.4f}")

    model.eval(); f1_metric.reset()
    with torch.no_grad():
        for batch in val_loader:
            ids  = batch["input_ids"].to(device)
            mask = batch["attention_mask"].to(device)
            y    = batch["labels"].to(device)
            preds = (torch.sigmoid(model(ids, mask)) > 0.5).int()
            f1_metric.update(preds, y.int())

    print("Val F1:", f1_metric.compute().item())

Train loss: 0.1163
Val F1: 0.5092906951904297


In [ ]:
!pip install -q streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.8 MB/s eta 0:00:00


In [ ]:
import torch

# Save model weights
torch.save(model.state_dict(), "sentiment_model.pt")

Creating a Streamlit App

In [ ]:
%%writefile app.py
import torch
import streamlit as st
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn

# Define emotion labels
emotion_labels = [
    "admiration", "amusement", "anger", "annoyance", "approval", "caring", "confusion",
    "curiosity", "desire", "disappointment", "disapproval", "disgust", "embarrassment",
    "excitement", "fear", "gratitude", "grief", "joy", "love", "nervousness", "optimism",
    "pride", "realization", "relief", "remorse", "sadness", "surprise", "neutral"
]

# Load tokenizer and model class
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

class DistilBERTModel(nn.Module):
    def __init__(self):
        super(DistilBERTModel, self).__init__()
        self.bert = AutoModel.from_pretrained("distilbert-base-uncased")
        self.classifier = nn.Linear(self.bert.config.hidden_size, len(emotion_labels))

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]  # [CLS] token
        return self.classifier(cls_output)

# Load trained weights
model = DistilBERTModel()
model.load_state_dict(torch.load("sentiment_model.pt", map_location=torch.device("cpu")))
model.eval()

# UI
st.title("🧠 Mental Health Sentiment & Emotion Detector")
user_input = st.text_area("Enter your sentence:", "")

if st.button("Analyze"):
    if user_input:
        inputs = tokenizer(user_input, return_tensors="pt", padding=True, truncation=True, max_length=128)
        with torch.no_grad():
            logits = model(inputs["input_ids"], inputs["attention_mask"])
            probs = torch.sigmoid(logits).squeeze()
            predicted = (probs > 0.5).nonzero(as_tuple=True)[0]

            if predicted.numel() == 0:
                st.info("No strong emotion detected.")
            else:
                results = [emotion_labels[i] for i in predicted]
                st.success(f"Detected emotions: {', '.join(results)}")
    else:
        st.warning("Please enter some text.")

Writing app.py


Launch Streamlit in Colab

In [ ]:
!ngrok config add-authtoken 2zgaPFRvILh4XTvvMy8efcouHWU_5gjGFRgL9HJM56nc8ApYj

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Step 1: Kill existing Streamlit (just in case)
!pkill streamlit

# Step 2: Import and reconnect ngrok (clean method)
from pyngrok import ngrok, conf

# Step 3: Manually specify tunnel to avoid YAML config error
conf.get_default().config_path = None  # ignore broken ngrok.yml
public_url = ngrok.connect(8501, bind_tls=True)

# Step 4: Print public URL
print(f"Streamlit app running at: {public_url}")

# Step 5: Launch your Streamlit app
!streamlit run app.py &>/dev/null &

Streamlit app running at: NgrokTunnel: "https://990b1f3f8e99.ngrok-free.app" -> "http://localhost:8501"


app.py

In [ ]:
code = '''
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Title
st.title("🧠 Mental Health Sentiment & Severity Detector")

# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=28)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# User input
text = st.text_area("Enter your mental health-related text here:")

# Predict
if st.button("Analyze"):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = torch.sigmoid(logits).squeeze()

    # Display results
    threshold = 0.5
    emotion_labels = [f"Label {i}" for i in range(28)]  # replace with actual emotion names if available
    detected = [(emotion_labels[i], float(probs[i])) for i in range(len(probs)) if probs[i] > threshold]

    if detected:
        st.subheader("Detected Emotions:")
        for emotion, score in detected:
            st.write(f"- {emotion} (Confidence: {score:.2f})")
    else:
        st.write("No strong emotion detected.")
'''

# Save to file
with open("app.py", "w") as f:
    f.write(code)

# Download it
from google.colab import files
files.download("app.py")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
reqs = '''
streamlit
transformers
torch
'''

with open("requirements.txt", "w") as f:
    f.write(reqs)

# Download
from google.colab import files
files.download("requirements.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
readme = '''
# 🧠 Mental Health Sentiment & Severity Detector

This project is a lightweight NLP-powered web app built using Streamlit and Hugging Face Transformers.

### 🔍 What It Does
- Detects emotions in mental health-related text.
- Uses DistilBERT for multi-label emotion classification.
- Outputs emotion severity using confidence scores.

### 🛠️ Built With
- Python
- PyTorch
- Hugging Face Transformers
- Streamlit

### 👨‍💻 Made by a passionate self-taught Data Scientist.
'''

with open("README.md", "w") as f:
    f.write(readme)

# Download
files.download("README.md")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>